# Maps of breadth of bootstrapped AA threshold confidence interval

In [1]:
import xarray as xr
from xarray.groupers import SeasonResampler
import numpy as np
import pandas as pd 
import os
import glob
from matplotlib import pyplot as plt
from cartopy import crs as ccrs
import cmocean

from funcs_support import get_params,get_filepaths, utility_print
from funcs_aux import get_landmask

In [2]:
from distributed import Client
# Start dask client
client = Client()
display(client)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 48,Total memory: 503.37 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46499,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 48
Started: Just now,Total memory: 503.37 GiB
Comm: tcp://127.0.0.1:45492,Total threads: 6
Dashboard: http://127.0.0.1:35600/status,Memory: 62.92 GiB
Nanny: tcp://127.0.0.1:37737,


In [ ]:
df = get_filepaths()
# df_tmp = df.query('experiment == "hindcasts" and run == "NMME"')
# df_tmp = df.query('model == "CHIRPS" and freq == "Amon"')
df_tmp = df.query('model == "GPCP"')
fn = df_tmp.iloc[0]['path']

ds = xr.open_dataset(fn,chunks = {'lat':20,'lon':20,'time':-1})

In [ ]:
# Trigger threshold
aa_trigger = 0.2

In [ ]:
# Get seasonal means
dsm = ds.groupby(
    time=SeasonResampler(["DJF", "MAM", "JJA", "SON"], drop_incomplete=True)
).mean()

In [ ]:
# Unstack into season x year
dsm['season'] = dsm.time.dt.season
dsm['year'] = dsm.time.dt.year
dsm = dsm.set_index(time=['season','year']).unstack()

In [ ]:
rng = np.random.default_rng()
# How many years / block
samp_size = 5
# How many bootstrap draws
ndraws = 1000
# Get start years of bootstrap blocks
start_years = rng.integers(dsm['year'].min(),dsm['year'].max() - samp_size,
                           (ndraws,np.ceil(dsm.sizes['year']/samp_size).astype(int)))

# Create bootstrap samples
draws = xr.concat([(dsm.sel(year = np.hstack([sy + np.arange(0,samp_size) for sy in sys])).
                    reset_index('year').drop_vars('year'))
           for sys in start_years],
              dim='draw')

draws = draws.chunk({'draw':10})

In [ ]:
# Get 20% trigger for each time series draw
threshs = draws.quantile(aa_trigger,dim=['year']).rename({'pr':'triggthresh'})

# Get range of threshholds
threshs['triggiqr'] = threshs['triggthresh'].quantile(0.975,dim='draw') - threshs['triggthresh'].quantile(0.025,dim='draw')

# Get means for reference
threshs['threshmean'] = threshs['triggthresh'].mean('draw')
threshs['prmean'] = draws.pr.mean('year').mean('draw')

# 
threshs['triggiqr_cv'] = threshs['triggiqr'] / threshs['triggthresh'].mean('draw')

In [ ]:
threshs = threshs.compute()